In [114]:
import pandas as pd
from sqlalchemy import create_engine, URL
from tqdm import tqdm

In [38]:
url_object = URL.create(
    drivername='postgresql+psycopg2',
    username='tie',
    password='TIE%2023!tuhh',
    host='134.28.58.100',
    port=45432,
    database='openalex_db',
)

In [39]:
engine = sqlalchemy.create_engine(url_object)
# Check if connection is successful
engine.connect()

In [42]:
df = pd.read_csv('/mnt/hdd01/patentsview/Reliance on Science - Cleantech Patents/df_oaid_Cleantech_Y02.csv')

In [46]:
# Convert column oaid and patent_id to string
df['oaid'] = df['oaid'].astype(str)
df['patent_id'] = df['patent_id'].astype(str)

In [49]:
# Erase everything in oaid and patent_id after the dot
df['oaid'] = df['oaid'].str.split('.').str[0]
df['patent_id'] = df['patent_id'].str.split('.').str[0]

In [68]:
# Add column 'full_oaid' with https://openalex.org/W + oaid
df['full_oaid'] = 'https://openalex.org/W' + df['oaid']

In [69]:
# Keep only columns oaid, patent_id and full_oaid
df = df[['oaid', 'patent_id', 'full_oaid']]

In [71]:
# Insert temporary table into database
df.to_sql('temp_table', engine, if_exists='replace', index=False)

145

In [74]:
# SQL query to get all data from data table that has an id in the temp_table
query = '''
    SELECT *
    FROM openalex.works 
    JOIN temp_table ON openalex.works.id = temp_table.full_oaid
'''

In [75]:
result = pd.read_sql(query, engine)

In [82]:
print(result.columns)

Index(['id', 'doi', 'title', 'display_name', 'publication_year',
       'publication_date', 'type', 'cited_by_count', 'is_retracted',
       'is_paratext', 'cited_by_api_url', 'abstract_inverted_index', 'oaid',
       'patent_id', 'full_oaid'],
      dtype='object')


In [83]:
print(result['abstract_inverted_index'].head())

0    {'IndexLength': 188, 'InvertedIndex': {'The': ...
1    {'IndexLength': 139, 'InvertedIndex': {'Sequen...
2    {'IndexLength': 144, 'InvertedIndex': {'Abstra...
3    {'IndexLength': 237, 'InvertedIndex': {'Edward...
4    {'IndexLength': 144, 'InvertedIndex': {'Abstra...
Name: abstract_inverted_index, dtype: object


In [99]:
print(result['abstract_inverted_index'].iloc[0])

{'IndexLength': 188, 'InvertedIndex': {'The': [0, 67, 144], 'Integrated': [1, 145], 'Solar': [2, 146], 'Upper': [3, 147], 'Stage': [4, 148], '(ISUS)': [5], 'Program': [6], 'at': [7], 'the': [8, 123, 173, 175, 179, 182], 'US': [9, 65], 'Air': [10, 25], 'Force`s': [11], 'Phillips': [12], 'Laboratory': [13], 'is': [14, 70, 142, 149], 'directing': [15], 'its': [16], 'efforts': [17], 'to': [18, 85, 88, 106, 163], 'solving': [19], 'endemic': [20], 'national': [21], 'spacelift': [22, 31], 'architecture': [23], 'problems.': [24], 'Force': [26], 'Space': [27], 'Command': [28], 'has': [29], 'called': [30], 'both': [32, 95], 'unaffordable': [33], 'and': [34, 47, 56, 98, 108, 112, 134, 159, 181, 185], 'nonresponsive;': [35], 'today`s': [36], 'space': [37, 138], 'launch': [38, 107], 'systems': [39], 'are': [40, 120, 167], 'descendants': [41], 'of': [42, 125, 140, 153], '1950`s': [43], 'era': [44], 'ICBM': [45], 'technologies': [46], 'require': [48, 109], 'prolonged': [49], 'on-pad': [50], 'processi

In [115]:
# Iterate over abstract_inverted_index column
for index, row in tqdm(result.iterrows()):
    word_index = []
    try:
        for key, value in row['abstract_inverted_index'].items():
            if key == 'InvertedIndex':
                for innerkey, innervalue in value.items():
                    for index in innervalue:
                        word_index.append([innerkey, index])
        # Sort list by index
        word_index.sort(key=lambda x: x[1])
        # Join first element of each list in word_index
        abstract = ' '.join([i[0] for i in word_index])
        # Add column abstract to result dataframe
        result.loc[index, 'abstract'] = abstract
        print(result.loc[index, 'abstract'])
        break
    except AttributeError:
        continue

0it [00:00, ?it/s]

The Integrated Solar Upper Stage (ISUS) Program at the US Air Force`s Phillips Laboratory is directing its efforts to solving endemic national spacelift architecture problems. Air Force Space Command has called spacelift both unaffordable and nonresponsive; today`s space launch systems are descendants of 1950`s era ICBM technologies and require prolonged on-pad processing. Advanced propulsion techniques, reusability, and modular architectures have not been generally incorporated into US systems. The ISUS system is an advanced, integrated upper stage concept that would permit payload realignment, allowing large payloads to be moved to smaller, less expensive boosters; such realignment both reduces cost and enhances responsiveness, as smaller boosters cost less to launch and require less vehicle and payload processing time. Three ISUS operational concepts are discussed, allowing the realignment of payloads currently launched aboard Titan IV/Centaur, Atlas IIAS, and Delta 2. A space demon

In [116]:
# Save result dataframe to json
result_test = result.head(100)
result_test.to_json('/home/thiesen/Documents/Cleantech_Concepts/oaid_test.json', orient='records')